In [ ]:
import numpy as np
from collections import Counter
import operator

from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import operator

class NaiveBayesClassifier:
    def __init__(self, alpha = 1):
        self.dic_data_index = dict()
        self.dic_data_by_class = dict()
        self.dic_data_posterior = dict()
        self.dic_data_prob_fitur = dict()
        self.prior = dict()
        self.class_ = 0
        self.alpha = alpha
        self.list_post = list()
        
    def train (self, X, y):
        self.X = X
        self.y = y
        
        vectorizer = CountVectorizer(binary=True)
        self.X = vectorizer.fit_transform(X).A
        self.fitur = vectorizer.get_feature_names()
        self.len_fitur = len(self.fitur)
        self.y = np.array(self.y)
        self.class_ = sorted(set(y))
        
        self.prior_class = dict(zip(Counter(self.y).keys(), Counter(self.y).values()))
        self.count_doc_c = self.prior_class.copy()
        for key, value in self.prior_class.items():
#             self.count_doc_c.update({key:len(self.y)})
            self.prior_class.update({key:value/len(self.y)})
        
        #menghitung jumlah class
        self.len_data = len(self.y)

        #mencari index data untuk class tertentu
#         self.dic_data_index = dict()
        for i in self.class_:
            isi_list = list()
            for index, j in enumerate(self.y):
        #         print(j)
                if i == j:
                    isi_list.append(index)
            self.dic_data_index.update({i:isi_list})
            
#         self.dic_data_by_class = dict()
        for key, value in self.dic_data_index.items():
            self.dic_data_by_class.update({key:(self.X[value])})
        # del self.dic_data_index
        
        self.count_fitur_c = dict()
        self.dic_data_prob_fitur = dict()
        for c, value in self.dic_data_by_class.items():
            count_per_doc = list()
            prob_fitur = list()
            sum_value  = value.sum()
            for ix, f in enumerate(value.transpose()):
                count_per_doc.append(f.sum())
                prob_fitur.append((f.sum()/(len(value)+self.len_fitur)))

            _dict_count_fitur = dict(zip(self.fitur, count_per_doc))
            bobot_fitur = dict(zip(self.fitur,prob_fitur))
            
            self.count_fitur_c.update({c:_dict_count_fitur})
            self.dic_data_prob_fitur.update({c:bobot_fitur})
            
    
    def predict(self, X):
        if self.alpha <= 0:
            raise Exception("alpha tidak boleh kurang dari atau sama dengan 0, alpha="+str(self.alpha)) 
        X = sorted(set(X.split()))
        try:
            self.dict_predict = dict()
            for c, value in self.dic_data_prob_fitur.items():

                bobot_fitur = list()
                for kata in X:
                    if kata in value:
                        bobot_fitur.append((self.count_fitur_c[c][kata]+self.alpha)/(self.count_doc_c[c]+self.len_fitur))
                        self.list_post.append("P({}|{})=({}+{})/({}+{} = {})"
                        .format(kata,c,self.count_fitur_c[c][kata],self.alpha,self.count_doc_c[c],self.len_fitur, (self.count_fitur_c[c][kata]+self.alpha)/(self.count_doc_c[c]+self.len_fitur)))
                  
                if len(bobot_fitur)>0:
                    post_prior = np.prod(bobot_fitur)*self.prior_class[c]
                    bobot_fitur_ = [str(round(x, 3)) for x in bobot_fitur]
                    self.list_post.append("P({}|{}) = {} x {} = {}".format(c,"X",self.prior_class[c], " x ".join(bobot_fitur_), post_prior,20))
                    self.list_post.append("===================================")
                    self.dict_predict.update({c:post_prior})
            return max(self.dict_predict.items(), key=operator.itemgetter(1))[0]
        except:
            print("err404or")
            return max(self.prior_class.items(), key=operator.itemgetter(1))[0]
